# <h1 style='Text-align: center;'>**Cleaning & Converting Data from a JSON file into a CSV**</h1>

`Created by: Erick Eduardo Robledo Montes`

---
---

<p style='Text-align: justify;'><i>Description:</i> Cleaning and converting data from the JSON file into a CSV format that is more user-friendly. The script performs various data cleaning tasks such as removing irrelevant columns, handling missing values, and converting data types to make the data more consistent and usable. The cleaned data is then exported to a CSV file, which can be easily imported into a spreadsheet or other data analysis tool. </p>

*Link: [https://secure.toronto.ca/nm/notices.json]*

In [1]:
import requests
import re
import pandas as pd
import numpy as np

## Classes
---
* The `DataScraper` class is responsible for retrieving and loading the JSON data from the specified URL. This class would likely contain methods for making a web request to the URL, parsing the JSON data, and loading it into a Python object.

* The `DataCleaner` class is responsible for cleaning and preprocessing the data before it is exported to CSV. This class would likely contain methods for removing irrelevant columns, handling missing values, and converting data types to make the data more consistent and usable.

* The `DataProcessor` class is responsible for coordinating the work of the DataScraper and DataCleaner classes, and for outputting the cleaned data to a CSV file. This class would likely contain methods for calling the appropriate methods from the DataScraper and DataCleaner classes, and for writing the cleaned data to a CSV file.

In [2]:
class DataScraper:
    """
    This class is responsible for scraping data from a website and
    converting it into a pandas DataFrame.
    """
    def __init__(self, url: str):
        """
        Initialize the DataScraper object with the specified URL.

        :param url: The URL of the website to scrape data from.
        """
        self.url = url
        self.data = None
        self.df = None

    def scrape_data(self):
        """
        Make a GET request to the website and store the JSON data in the
        'data' attribute.
        """
        response = requests.get(self.url)
        if response.status_code == 200:
            self.data = response.json()
            print(f"Success. Status code:", response.status_code)
        else:
            print(f"Error. Status code:", response.status_code)

    def convert_data_to_df(self, column_name: str):
        """
        Convert the JSON data stored in the 'data' attribute into a pandas
        DataFrame and store it in the 'df' attribute.
        """
        for key in self.data.keys():
            self.data[key] = pd.Series(self.data[key])
        # The column "Records" stored multiple columns to work with
        data_list = [self.data[column_name][i] for i in range(len(self.data[column_name]))]
        self.df = pd.concat([pd.DataFrame.from_dict(d, orient='index').T for d in data_list], ignore_index=True)

class DataCleaner:
    """
    This class is responsible for cleaning the data in a pandas DataFrame.
    """
    def __init__(self, df: pd.DataFrame):
        """
        Initialize the DataCleaner object with the specified DataFrame.

        :param df: The DataFrame to clean.
        """
        self.df = df

    def clean_data(self):
        """
        Perform data cleaning operations on the DataFrame.
        """
        # Remove non-integer values from the column 'noticeId'
        self.df = self.df[self.df['noticeId'].apply(lambda x: isinstance(x, int))]
        # Sort the 'noticeId' column in ascending order
        self.df.sort_values(by='noticeId', ascending=True, inplace=True)
        # Drop column who only has empty lists
        self.df.drop(columns='planningApplicationNumbers', inplace=True)
        # Transform the column 'noticeDate' to datetime
        self.df['noticeDate'] = pd.to_datetime(self.df['noticeDate'], unit='ms')
        # Remove HTML tags using regular expressions
        self.df['noticeDescription'] = self.df['noticeDescription'].apply(lambda x: re.sub('<[^<]+?>', '', x))
        # Remove NaN values
        self.df["accessibilityDescription"].replace(np.nan, '', inplace=True)
        self.df["decisionBody"].replace(np.nan, '', inplace=True)
        # Call the 'clean_column' function to keep going on data cleaning
        self.clean_column("contact")
        self.clean_column("eventList")
        self.clean_column("backgroundInformationList") 
        self.clean_column("addressList")
        self.clean_column("otherReferenceList")
        self.remove_data_from_column("uniqueMapUrl")
        self.remove_data_from_column("topics")
    
    def clean_column(self, column_name: str):
        """
        Perform common cleaning operations on a specified column.
        :param column_name: The name of the column to clean.
        """
        column_data = []
        if column_name == "contact":
            column_list = list(self.df[column_name])
            for i, val in enumerate(column_list):
                if pd.isnull(val):
                    column_list[i] = {}
            column_data = [x for x in column_list if type(x)==dict]
        else:
            column_list = [self.df[column_name][i] for i in range(len(self.df))] 
            for i in range(len(self.df)):
                if column_list[i] == []:
                    column_data.append({})
                else:
                    column_data.append(column_list[i][0])

        # Create the DataFrame
        column_df = pd.DataFrame(column_data)
        # Convert all elements to strings before joining
        column_df = column_df.astype(str)
        # Join the elements of each row in the DataFrame
        self.df[column_name] = column_df.apply(lambda x: ", ".join(x), axis=1)
        while True:
            self.df[column_name] = self.df[column_name].str.replace('^nan, |, nan, | nan |nan|, nan', '', regex=True)
            if ', nan' not in self.df[column_name].to_string():
                break
    
    def remove_data_from_column(self, column_name: str):
        for index, row in self.df.iterrows():
            if row[column_name]:
                self.df.at[index, column_name] = row[column_name][0]
            else:
                self.df.at[index, column_name] = ''

    def optimize_data(self):
        """
        Perform optimization on the DataFrame.
        """
        self.df = self.df.drop_duplicates()
        self.df = self.df.reset_index(drop=True)

class DataProcessor:
    """
    This class is responsible for processing the cleaned data.
    """
    def __init__(self, df: pd.DataFrame):
        """
        Initialize the DataProcessor object with the specified DataFrame.
        :param df: The cleaned DataFrame to process.
        """
        self.df = df
        
    def export_data(self, file_name: str):
        """
        Export the processed DataFrame to a CSV file.
        :param file_name: The name of the CSV file to export the data to.
        """
        self.df.to_csv(file_name, index=None, header=True, encoding='utf-8-sig')

## Processing JSON Data into a CSV File Using Python Classes
---
* The first three lines of the code set the URL of the website to scrape, the path and file name for the exported CSV file, and the name of the column in the JSON data that contains the relevant records.

* The script then creates an instance of the DataScraper class and uses its `scrape_data()` method to retrieve the data from the website. The `convert_data_to_df()` method is used to convert the JSON data into a DataFrame.

* Next, an instance of the DataCleaner class is created and its `clean_data()` method is used to clean the data in the DataFrame. The `optimize_data()` method is used to optimize the data.

* Finally, an instance of the DataProcessor class is created and its `export_data()` method is used to export the processed data to a CSV file. The path and file name for the exported CSV file are passed as an argument.

* At the end, the script prints "Data processing completed." to indicate that the data scraping, cleaning, and conversion process has been completed successfully.

In [3]:
url = "https://secure.toronto.ca/nm/notices.json"
path = "dataset/"
file_name = "processed_data.csv"
column_name = "Records"

# Initialize the DataScraper object
scraper = DataScraper(url)
# Scrape data from the website
scraper.scrape_data()
# Convert the JSON data into a DataFrame
scraper.convert_data_to_df(column_name)

# Initialize the DataCleaner object
cleaner = DataCleaner(scraper.df)
# Clean the data in the DataFrame
cleaner.clean_data()
cleaner.optimize_data()

# Initialize the DataProcessor object
processor = DataProcessor(cleaner.df)
# Export the processed data to a CSV file
processor.export_data(path+file_name)

print("Data processing completed.")

Success. Status code: 200
Data processing completed.
